In [1]:
!pip install -q transformers

In [2]:
import torch
import sklearn
import numpy as np
import pandas as pd
from torch import nn
from sklearn import metrics
from tqdm.notebook import tqdm
from transformers import AdamW
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.utils.class_weight import compute_class_weight

In [3]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev

device(type='cpu')

In [4]:
class PatDataset(Dataset):
    def __init__(self, px, dx, rx, static_features, y):
        self.px = px
        self.dx = dx
        self.rx = rx
        self.s = static_features
        self.y = y
            
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.px[idx], self.dx[idx], self.rx[idx], self.s[idx], self.y[idx]


class MyModel(nn.Module):
    def __init__(self, path=None):
        super().__init__()
    
        self.px_seq = nn.Sequential(
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.LayerNorm(128, eps=1e-12, elementwise_affine=True)
            )
        self.dx_seq = nn.Sequential(
            nn.Linear(768, 128),
            nn.ReLU(),
            nn.LayerNorm(128, eps=1e-12, elementwise_affine=True)
            )
        self.rx_seq = nn.Sequential(
            nn.Linear(806, 128),
            nn.ReLU(),
            nn.LayerNorm(128, eps=1e-12, elementwise_affine=True)
            )
        self.static_seq = nn.Sequential(
            nn.Linear(182, 64),
            nn.ReLU(),
            nn.LayerNorm(64, eps=1e-12, elementwise_affine=True)
            )
        self.concat_seq = nn.Sequential(
            nn.Linear(448, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 2),  
            )

    def forward(self, px_embed, dx_embed, rx_embed, static_embed):
        px_features = self.px_seq(px_embed)
        dx_features = self.px_seq(dx_embed)
        rx_features = self.rx_seq(rx_embed)
        static_features = self.static_seq(static_embed)
        concat = torch.cat((px_features, dx_features, rx_features, static_features), dim=1)
        return self.concat_seq(concat)


class Trainer:
    def __init__(self, train_data, val_data):
        self.train_data = train_data
        self.val_data = val_data
        if torch.cuda.is_available():
            self.dev = torch.device('cuda')
        else:
            self.dev = torch.device('cpu')
        print("Using: ", self.dev)

    @staticmethod
    def _roc_auc(y_true, y_raw_logits, class_idx=1):
        y_score = F.softmax(y_raw_logits, dim=1).cpu().detach().numpy()[:, class_idx]
        return metrics.roc_auc_score(y_true.cpu().detach().numpy(), y_score)

    @staticmethod
    def _accuracy(true, pred_proba, class_idx=None):
        preds = torch.argmax(pred_proba, dim=1)
        if class_idx is None:
            score = (true == preds).float().mean()
        else:
            score = (true==preds)[true==class_idx].float().mean()
        return score

    @staticmethod
    def _get_optimizer_with_decay(model, lr):
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': 0.01, 'lr':lr},
            {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
             'weight_decay': 0.0, 'lr':lr}
             ]
        return AdamW(optimizer_grouped_parameters, lr=lr)

    def _get_loss_func(self, class_wts=None):
        if class_wts is not None:
            loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_wts).type(torch.float).to(self.dev))
        else:
            loss_fn = torch.nn.CrossEntropyLoss()
        return loss_fn
    
    @staticmethod
    def get_class_wts(y):
        return compute_class_weight('balanced', np.unique(y), y)

    def train_model(self, model, n_epochs=1, lr=1e-5, class_wts=None,
                    path_to_save_model='./model.tar', logging_step=5,
                    patience=3):
        loss_fn = self._get_loss_func(class_wts=class_wts)
        optimizer = Trainer._get_optimizer_with_decay(model, lr)

        model.to(self.dev)
        trn_loss = []
        val_loss = []
        val_accu = []
        max_val_accu = 0
        count = 0
        for epoch in range(n_epochs):
            print(f"Epoch: {epoch}")
            trn_loss_per_epoch = []
            model.train()
            for i, batch in enumerate(tqdm(self.train_data)):
                batch = [x.to(self.dev) for x in batch]
                outputs = model(*batch[:-1])
                loss = loss_fn(outputs, batch[-1])
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                del batch
                torch.cuda.empty_cache()
                trn_loss_per_epoch.append(float(loss))
                if i % logging_step == 0:
                    print("train_loss: ", np.mean(trn_loss_per_epoch))
            trn_loss.append(np.mean(trn_loss_per_epoch))
            
            model.eval()
            valid_loss_per_epoch = []
            yhat_lst = []
            y_lst = []
            with torch.no_grad():
                for batch in tqdm(self.val_data):
                    batch = [x.to(self.dev) for x in batch]
                    outputs = model(*batch[:-1])
                    valid_loss_per_epoch.append(float(loss_fn(outputs, batch[-1])))
                    yhat_lst.append(outputs)
                    y_lst.append(batch[-1])   
                tmp = Trainer._roc_auc(torch.cat(y_lst, dim=0), torch.cat(yhat_lst, dim=0))
            val_loss.append(np.mean(valid_loss_per_epoch))
            val_accu.append(tmp)
            
            print(f"Training Loss for epoch {epoch}: ", trn_loss[-1])
            print("Validation Loss: ", val_loss[-1], "| Validation roc_auc: ", val_accu[-1])

            if val_accu[-1] > max_val_accu:
                count = 0
                max_val_accu = val_accu[-1]
                torch.save({
                            'epoch': epoch,
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'training_loss': trn_loss[-1],
                            'val_loss': val_loss[-1],
                            'val_accuracy': val_accu[-1]
                            }, path_to_save_model)
                print(f"Checkpoint saved!")
            else:
                count = count + 1

            if count > patience:
                print("Stopping early, restoring best checkpoint..")
                model_parameters = torch.load(path_to_save_model, map_location = self.dev)
                model.load_state_dict(model_parameters['model_state_dict'])
                optimizer.load_state_dict(model_parameters['optimizer_state_dict'])
                break 
            print("Max roc_auc till now: ", max_val_accu)
        return model.eval(), optimizer

In [5]:
px_trn = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/trn_px_journey.pkl")
dx_trn = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/trn_dx_journey.pkl")
rx_trn = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/imp_rx_trn.pkl")
y_trn = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/y_trn.pkl")
rx_trn.shape

(23868, 806)

In [6]:
px_val = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/val_px_journey.pkl")
dx_val = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/val_dx_journey.pkl")
rx_val = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/imp_rx_val.pkl")
y_val = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/y_val.pkl")
rx_val.shape

(4212, 806)

In [7]:
px_test = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/test_px_journey.pkl")
dx_test = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/test_dx_journey.pkl")
rx_test = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/imp_rx_test.pkl")
y_test = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/y_test.pkl")
rx_test.shape

(7020, 806)

In [8]:
trn_static = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/trn_static.pkl")
val_static = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/val_static.pkl")
test_static = pd.read_pickle("/content/drive/MyDrive/ColabData/datasets/MIMIC-3/PatientLevel/test_static.pkl")
trn_static.shape, val_static.shape, test_static.shape

((23868, 182), (4212, 182), (7020, 182))

In [9]:
bs = 1024

trn_ds = PatDataset(px_trn, dx_trn, rx_trn.values, trn_static.values, y_trn['switch_flag'].tolist())
trn_dl = torch.utils.data.DataLoader(trn_ds, batch_size=bs, num_workers=0, pin_memory=False)

val_ds = PatDataset(px_val, dx_val, rx_val.values, val_static.values, y_val['switch_flag'].tolist())
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=bs, num_workers=0, pin_memory=False)

test_ds = PatDataset(px_test, dx_test, rx_test.values, test_static.values, y_test['switch_flag'].tolist())
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs, num_workers=0, pin_memory=False)

In [10]:
class_wts = compute_class_weight('balanced', np.unique(y_trn['switch_flag'].values), y_trn['switch_flag'].values)
class_wts

array([ 0.51734004, 14.9175    ])

In [11]:
trainer = Trainer(trn_dl, val_dl)

Using:  cpu


In [12]:
model = MyModel()

model, optimizer = trainer.train_model(
    model, n_epochs=200, lr=1e-5, class_wts=class_wts,
    path_to_save_model='/content/drive/MyDrive/ColabData/saved_models/PatientBERT/only_top/sepsis-readmission-only-top-with-static-features.tar',
    logging_step=46, patience=10
    )

Epoch: 0


train_loss:  0.6937960386276245




Training Loss for epoch 0:  0.6951270923018456
Validation Loss:  0.6925402522087097 | Validation roc_auc:  0.5039511437934117
Checkpoint saved!
Max roc_auc till now:  0.5039511437934117
Epoch: 1


train_loss:  0.6898974180221558




Training Loss for epoch 1:  0.691310927271843
Validation Loss:  0.6893953561782837 | Validation roc_auc:  0.550254263420039
Checkpoint saved!
Max roc_auc till now:  0.550254263420039
Epoch: 2


...


Training Loss for epoch 72:  0.46213118607799214
Validation Loss:  0.6277498245239258 | Validation roc_auc:  0.7010998047075753
Max roc_auc till now:  0.7020091949457414
Epoch: 73


train_loss:  0.5063905715942383




Training Loss for epoch 73:  0.4579450649519761
Validation Loss:  0.6294668197631836 | Validation roc_auc:  0.7007600899634328
Max roc_auc till now:  0.7020091949457414
Epoch: 74


train_loss:  0.5019139647483826




Training Loss for epoch 74:  0.4537044018507004
Validation Loss:  0.631260085105896 | Validation roc_auc:  0.7006451095884922
Stopping early, restoring best checkpoint..


In [13]:
model_parameters = torch.load(
    "/content/drive/MyDrive/ColabData/saved_models/PatientBERT/only_top/sepsis-readmission-only-top-with-static-features.tar",
    map_location = dev
    )
model.load_state_dict(model_parameters['model_state_dict'])
model = model.eval()

In [14]:
dl = trn_dl
yhat_lst = []
y_lst = []
with torch.no_grad():
    for batch in tqdm(dl):
        yhat = model(*batch[:-1])
        yhat_lst.append(yhat)
        y_lst += list(batch[-1].cpu().numpy())

print(Trainer._roc_auc(torch.tensor(y_lst), torch.cat(yhat_lst, dim=0)))


0.8553900966707126


In [15]:
dl = val_dl
yhat_lst = []
y_lst = []
with torch.no_grad():
    for batch in tqdm(dl):
        yhat = model(*batch[:-1])
        yhat_lst.append(yhat)
        y_lst += list(batch[-1].cpu().numpy())

print(Trainer._roc_auc(torch.tensor(y_lst), torch.cat(yhat_lst, dim=0)))


0.7020091949457414


In [16]:
dl = test_dl
yhat_lst = []
y_lst = []
with torch.no_grad():
    for batch in tqdm(dl):
        yhat = model(*batch[:-1])
        yhat_lst.append(yhat)
        y_lst += list(batch[-1].cpu().numpy())

print(Trainer._roc_auc(torch.tensor(y_lst), torch.cat(yhat_lst, dim=0)))


0.709803540350272


# **XGBoost** 

## Without Static

In [17]:
from xgboost import XGBClassifier

In [23]:
data_trn = pd.concat([pd.DataFrame(px_trn), pd.DataFrame(dx_trn), rx_trn.reset_index(drop=True)], axis=1)
data_val = pd.concat([pd.DataFrame(px_val), pd.DataFrame(dx_val), rx_val.reset_index(drop=True)], axis=1)
data_test = pd.concat([pd.DataFrame(px_test), pd.DataFrame(dx_test), rx_test.reset_index(drop=True)], axis=1)

In [24]:
clf = XGBClassifier(n_estimators=36,
                    scale_pos_weight=(y_trn.shape[0]-800)/y_trn['switch_flag'].sum(),
                    verbosity=2)
clf = clf.fit(data_trn.values, y_trn['switch_flag'].tolist(),
              early_stopping_rounds=10,
              eval_set=[(data_val.values, y_val['switch_flag'].tolist())],
              eval_metric='auc')

[15:15:11] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[0]	validation_0-auc:0.630452
Will train until validation_0-auc hasn't improved in 10 rounds.
[15:15:13] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[1]	validation_0-auc:0.636731
[15:15:16] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[2]	validation_0-auc:0.658897
[15:15:19] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[3]	validation_0-auc:0.664356
[15:15:21] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[4]	validation_0-auc:0.668267
[15:15:24] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[5]	validati

In [25]:
yhat = clf.predict_proba(data_trn.values)
metrics.roc_auc_score(y_trn['switch_flag'].tolist(), yhat[:, 1])

0.7680008561643834

In [26]:
yhat = clf.predict_proba(data_val.values)
metrics.roc_auc_score(y_val['switch_flag'].tolist(), yhat[:, 1])

0.6757788613807052

In [27]:
yhat = clf.predict_proba(data_test.values)
metrics.roc_auc_score(y_test['switch_flag'].tolist(), yhat[:, 1])

0.6605879678264005

## With Static

In [ ]:
data_trn = pd.concat([pd.DataFrame(px_trn), pd.DataFrame(dx_trn), rx_trn.reset_index(drop=True), trn_static.reset_index(drop=True)], axis=1)
data_val = pd.concat([pd.DataFrame(px_val), pd.DataFrame(dx_val), rx_val.reset_index(drop=True), val_static.reset_index(drop=True)], axis=1)
data_test = pd.concat([pd.DataFrame(px_test), pd.DataFrame(dx_test), rx_test.reset_index(drop=True), test_static.reset_index(drop=True)], axis=1)

In [ ]:
clf = XGBClassifier(n_estimators=36, scale_pos_weight=(y_trn.shape[0]-800)/y_trn['switch_flag'].sum(), verbosity=2)
clf = clf.fit(data_trn.values, y_trn['switch_flag'].tolist(), early_stopping_rounds=10, eval_set=[(data_val.values, y_val['switch_flag'].tolist())], eval_metric='auc')

[15:02:34] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[0]	validation_0-auc:0.671095
Will train until validation_0-auc hasn't improved in 10 rounds.
[15:02:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[1]	validation_0-auc:0.685139
[15:02:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[2]	validation_0-auc:0.701385
[15:02:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[3]	validation_0-auc:0.695175
[15:02:45] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[4]	validation_0-auc:0.702331
[15:02:47] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[5]	validati

In [ ]:
yhat = clf.predict_proba(data_trn.values)
metrics.roc_auc_score(y_trn['switch_flag'].tolist(), yhat[:, 1])

0.7862989585139588

In [ ]:
yhat = clf.predict_proba(data_val.values)
metrics.roc_auc_score(y_val['switch_flag'].tolist(), yhat[:, 1])

0.7188599173186576

In [ ]:
yhat = clf.predict_proba(data_test.values)
metrics.roc_auc_score(y_test['switch_flag'].tolist(), yhat[:, 1])

0.7246382037974881